# Lambda Iterations

In [1]:
import os
import sys
import matplotlib.pyplot as plt

magritteFolder = '/home/atulits/atulit/MasterThesis/Magritte/'

sys.path.append(magritteFolder)

import numpy             as np
import matplotlib.pyplot as plt
import magritte.setup    as setup
import magritte.core     as magritte

In [2]:
dimension = 1
npoints   = 100
nrays     = 200
nspecs    = 5
nlspecs   = 1
nquads    = 15

r_in   = 1.0E13   # [m]
r_out  = 7.8E16   # [m]
nH2_in = 2.0E13   # [m^-3]
X_mol  = 1.0E-6   # [.]
temp   = 20.0     # [K]
turb   = 150.00   # [.]


def nH2 (r):
    return nH2_in * np.power(r_in/r, 2.0)

def nTT (r):
    return X_mol  * nH2(r)


rs = np.logspace (np.log10(r_in), np.log10(r_out), npoints, endpoint=True)


modelFile = 'vanZadelhoff_1a_1D.hdf5'
lamdaFile = f'{magritteFolder}tests/data/test.txt'


model = magritte.Model ()
model.parameters.set_spherical_symmetry(True)
model.parameters.set_pop_prec          (1.0e-6)
model.parameters.set_model_name        (modelFile)
model.parameters.set_dimension         (dimension)
model.parameters.set_npoints           (npoints)
model.parameters.set_nrays             (nrays)
model.parameters.set_nspecs            (nspecs)
model.parameters.set_nlspecs           (nlspecs)
model.parameters.set_nquads            (nquads)

model.geometry.points.position.set([[r, 0, 0] for r in rs])
model.geometry.points.velocity.set(np.zeros((npoints, 3)))

model.chemistry.species.abundance = [[     0.0, nTT(r), nH2(r),  0.0,      1.0] for r in rs]
model.chemistry.species.symbol    =  ['dummy0', 'test',   'H2', 'e-', 'dummy1']

model.thermodynamics.temperature.gas  .set( temp                 * np.ones(npoints))
model.thermodynamics.turbulence.vturb2.set((turb/magritte.CC)**2 * np.ones(npoints))

model = setup.set_Delaunay_neighbor_lists (model)
model = setup.set_Delaunay_boundary       (model)
model = setup.set_boundary_condition_CMB  (model)
model = setup.set_rays_spherical_symmetry (model)
model = setup.set_linedata_from_LAMDA_file(model, lamdaFile)
model = setup.set_quadrature              (model)

model.write()
model.read ()

model.compute_spectral_discretisation ()
model.compute_inverse_line_widths     ()
model.compute_LTE_level_populations   ()

/home/atulits/atulit/MasterThesis/Magritte/src/io/python/io_python_hdf5.py:83: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  with hp.File (io_file) as file:
/home/atulits/atulit/MasterThesis/Magritte/src/io/python/io_python_hdf5.py:132: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  with hp.File (io_file) as file:


0

In [3]:
def max_relative_difference(a, b):
    return np.max(2.0 * np.abs((a - b) / (a + b)))

In [4]:
def Anderson_acc_gen(x , num_it, l):
    unit = np.ones(num_it-1)
    #Column vectors for U matrix but here we have row matrix
    U_transpose = []
    for i in range(0,num_it-1):
            U_transpose.append((x[i] - x[i+1]))
    U_transpose = np.array(U_transpose)  
    
    #multiply two matrix
    M = np.matmul(U_transpose, U_transpose.T)
    
    # l is regularisation coefficient  
    # inverse of U and U_T
    M = np.linalg.inv(M + l*np.identity(len(M)))
    # Normalizaton factor
    Norm = np.linalg.multi_dot([unit,M,unit])
    #coeeficients c's
    c = np.matmul(M,unit)
    c = c/Norm
    print(sum(c))
    
    S_acc = 0
    for p in range(0,num_it-1):
        S_acc = S_acc + (c[p]*x[p+1])  
    return (S_acc,c)
    

In [ ]:
Nmax = 100

nlev = model.lines.lineProducingSpecies[0].linedata.nlev
abn = np.array(model.chemistry.species.abundance, dtype=np.float64)[:,1]

it_diff = []
pops = []


#LTE Level population
pop  = np.copy(model.lines.lineProducingSpecies[0].population)
pops.append((pop.reshape((npoints,nlev)).T/abn).T.ravel())

#order of Anderson acc and regularization coefficient
m = 1
l = 0
Coefficients = []

for it in range(Nmax):
    # Compute radiation field and resulting populations
    if  it>=2: 
        A = []
        num_iterates = (m+2)
        for p in range(0,num_iterates):
            A.append(np.array(pops[p] , dtype = np.float64))
        print(len(A))
        S_acc , c = Anderson_acc_gen(A, num_iterates , l)
        
        Coefficients.append(c)
        
        pot=(S_acc.reshape((npoints,nlev)).T*abn).T.ravel()
        np.save("levelpop"+str(it),pot)
        #plt.plot(np.arange(len(pot)) , pot)
        #plt.yscale("log")
        #plt.savefig("levelpop"+str(it)+".png")
        #plt.show()
        
        model.lines.lineProducingSpecies[0].population = (S_acc.reshape((npoints,nlev)).T*abn).T.ravel()
        model.lines.set_emissivity_and_opacity()
        model.compute_radiation_field()
        model.compute_Jeff()
        model.compute_level_populations_from_stateq()
        
        # Compute the maximum relative change
        pop = np.copy(model.lines.lineProducingSpecies[0].population)

        m = m+1
    else:
        model.compute_radiation_field()
        model.compute_Jeff()
        model.compute_level_populations_from_stateq()
        # Compute the maximum relative change
        pop = np.copy(model.lines.lineProducingSpecies[0].population)
    
    pops.append((pop.reshape((npoints,nlev)).T/abn).T.ravel())

    max_diff = max_relative_difference(pops[-1], pops[-2])
    it_diff.append(max_diff)
    # Check for convergence
    if (max_diff < 1.0e-6):
        break
    print('After', it, 'iteration(s), max_diff =', max_diff)

After 0 iteration(s), max_diff = 0.6906639565036251066
After 1 iteration(s), max_diff = 1.3868983562409522291
3
1.0
After 2 iteration(s), max_diff = 1.2173526160324879961
4
1.0
After 3 iteration(s), max_diff = 1.4476575444698022956
5
1.0
After 4 iteration(s), max_diff = 0.058575645570806364895
6
1.0
After 5 iteration(s), max_diff = 1.3733913523538798519
7
1.0
After 6 iteration(s), max_diff = 0.01561430179401491483
8
1.0
After 7 iteration(s), max_diff = 1.0491700251112940218
9
1.0
After 8 iteration(s), max_diff = 0.20583332229415174788
10
1.0
After 9 iteration(s), max_diff = 0.12533957386025559097
11
0.9999999999999998
After 10 iteration(s), max_diff = 0.16109049140136004565
12
1.0
After 11 iteration(s), max_diff = 0.27198489509027982898
13
0.9999999999999998
After 12 iteration(s), max_diff = 0.21860053557465666264
14
1.0000000000000002
After 13 iteration(s), max_diff = 0.124282966482208332455
15
1.0
After 14 iteration(s), max_diff = 1.1189802843996794142
16
0.9999999999999999
After 15 

In [ ]:
np.save("Lambda_online_it",pops)
plt.plot(np.arange(len(it_diff)) , it_diff)
plt.yscale("log")
plt.savefig("plot.png")
plt.show()


In [ ]:
l_norm = []
for c in Coefficients:
    size = c[0] - np.sum(c[1:])
    l_norm.append(size**2) 
    print(c)
plt.stem(np.arange(len(l_norm)) , l_norm , 'ro')

In [ ]:
pops_lambda  = np.load("Lambda_it.npy")
print(pops_lambda[-1])